In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
#data processing
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb

#algorithms
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score, recall_score, accuracy_score
import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Dense

#dataframe display settings
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 50)

In [3]:
#Plotly Libraris
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.colors import n_colors
from plotly.subplots import make_subplots
# Minmax scaler
from sklearn.preprocessing import MinMaxScaler

## Data Preprocessing
After saving the training and testing dataset in "train_data" and "test_data" dataframes repectively, we'll replace the missing values in "Age" and "Fare" columns with the average values of the respective columns.

In [4]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")

# Fill missing values in 'Age' with the mean from the training data
train_data['Age'] = train_data['Age'].fillna(train_data['Age'].mean())
test_data['Age'] = test_data['Age'].fillna(train_data['Age'].mean())

# Convert 'Age' to float data type
train_data['Age'] = train_data['Age'].astype(float)
test_data['Age'] = test_data['Age'].astype(float)

# Fill missing values in 'Fare' with the mean from the training data
train_data['Fare'] = train_data['Fare'].fillna(train_data['Fare'].mean())
test_data['Fare'] = test_data['Fare'].fillna(train_data['Fare'].mean())

# Convert 'Fare' to float data type
train_data['Fare'] = train_data['Fare'].astype(float)
test_data['Fare'] = test_data['Fare'].astype(float)

train_data.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
#women survival 
women = train_data[train_data.Sex == 'female']["Survived"]
#men survival 
men = train_data[train_data.Sex == 'male']["Survived"]
print("Survival rate for women is {:.2f} and for men is {:.2f}".format((sum(women)/len(women))*100, (sum(men)/len(men))*100))

Survival rate for women is 74.20 and for men is 18.89


In [6]:
data = train_data
data['Died'] = 1 - data['Survived']

## Data Exploration/Analysis

**1. Gender and no. of Passangers**

In [7]:
fig = go.Figure(data=[
    go.Bar(name='Survived', x=data['Sex'], y=[data[data.Sex==data['Sex'][0]]['Survived'].agg(sum), data[data.Sex==data['Sex'][1]]['Survived'].agg(sum)], marker_color='#EB89B5', opacity=0.75),
    go.Bar(name='Died', x=data['Sex'], y=[data[data.Sex==data['Sex'][0]]['Died'].agg(sum), data[data.Sex==data['Sex'][1]]['Died'].agg(sum)], marker_color='#330C73', opacity=0.75)
])
fig.update_layout(barmode='stack', width=600, title_text='Survivors categorized as Male/Female', xaxis_title_text='Sex', yaxis_title_text='Number of passangers')
fig.show()

**2. Survival based on Fare, Embarked and Pclass**

In [8]:
facet_data = data[['Sex', 'Survived', 'Embarked', 'Pclass', 'Fare']].groupby(['Sex', 'Survived', 'Embarked', 'Pclass']).agg('sum').reset_index()
fig  = px.bar(facet_data, x='Sex', y='Fare', facet_row='Survived', facet_col='Pclass', color='Embarked', barmode='group')
fig.update_layout(title_text='Different view of Passenger survival with respect to Fare, Embarked and Pclass')
fig.show()

**3. Survival by Age of passangers**

In [9]:
fig = go.Figure()
fig.add_trace(go.Violin(x=data['Sex'][data['Survived']==1], y=data['Age'][data['Survived']==1],
                         name='Survived', side='negative', legendgroup='Survived', scalegroup='Survived', line_color='blue'))
fig.add_trace(go.Violin(x=data['Sex'][data['Died']==1], y=data['Age'][data['Died']==1],
                         name='Died', side='positive', legendgroup='Died', scalegroup='Died', line_color='dark orange'))
fig.update_traces(meanline_visible=True, width=1)
fig.update_layout(violingap=0, violinmode='overlay', width=700, title_text='Survivors categorized as Male/Female related to their Age', xaxis_title_text='Sex', yaxis_title_text='Age of passangers')
fig.show()

**4. Survival by Gender and Age of pasangers**

In [10]:
data['Survived'] = data['Survived'].astype(str)
fig_scatter = px.scatter(data, x='Fare', y='Age', color='Survived', facet_col='Sex')
fig_scatter.update_layout(width=800, title_text='Survivors categorized as Male/Female', yaxis_title_text='Age of passangers')
fig_scatter.show()

data['Survived'] = data['Survived'].astype(int)

**5. Survival by Fare**

In [11]:
fig_hist = go.Figure()
fig_hist.add_trace(go.Histogram(x=data[data['Survived']==1]['Fare'], name='Survived', marker_color='#EB89B5', opacity=0.75))
fig_hist.add_trace(go.Histogram(x=data[data['Died']==1]['Fare'], name='Died', marker_color='#330C73', opacity=0.75))
fig_hist.update_layout(barmode='stack', title_text='Histogram representing relation between Survivor/Dead with Fare',
                       xaxis_title_text='Fare', yaxis_title_text='Number of Passangers', bargap=0.2)
fig_hist.show()

**6. Survival by Age and Fare**

In [12]:
fig_scatter_plane = go.Figure()
fig_scatter_plane.add_trace(go.Scatter(x=data[data['Survived']==1]['Age'], y=data[data['Survived']==1]['Fare'],
                                       mode='markers', name='Survived', marker=dict(size=data[data['Survived']==1]['Fare']/8)))
fig_scatter_plane.add_trace(go.Scatter(x=data[data['Died']==1]['Age'], y=data[data['Died']==1]['Fare'],
                                       mode='markers', name='Died', marker=dict(size=data[data['Died']==1]['Fare']/8)))
fig_scatter_plane.update_layout(title_text='Scatter representing relation between Fare and Age of passangers',
                       xaxis_title_text='Age', yaxis_title_text='Fare')
fig_scatter_plane.show()

**7. Average Fare for each Pclass**

In [14]:
fig_bar_avgfare = go.Figure()
fig_bar_avgfare.add_trace(go.Bar(x=np.sort(data['Pclass'].unique()), y=data.groupby('Pclass').mean(numeric_only=True)['Fare'], marker_color=np.sort(data['Pclass'].unique())))
fig_bar_avgfare.update_traces(width=0.5)
fig_bar_avgfare.update_layout(width=600, title_text='Bar chart representing Class and Avg. Fare relation',
                       xaxis_title_text='Pclass', yaxis_title_text='Avg. Fare')
fig_bar_avgfare.show()

## Creating New Categories

In [16]:
# Creating combined dataframe
x_train = train_data.drop(['Survived', 'Died'], axis=1)
y_train = train_data['Survived']
x_test = test_data

# Combining the dataframes
df_combined = pd.concat([x_train, x_test], axis=0)
df_combined.shape

(1309, 11)

## 1. Creating categories based on Family size.
We'll calculate the family size for each passanger based on the values of "Parch" and "SibSp" and then categorize the individuals based on the value of family size into, Singleton(for family_size=1), SmallFamily(for family_size=2 to 4) and largeFamily(for family_size=5 to 8).

In [17]:
#creating funtion for creating categories based on family size
def family_size():
    global df_combined
    df_combined['FamilySize'] = df_combined['Parch'] + df_combined['SibSp'] + 1
    #feature to decide family size
    df_combined['Singleton'] = df_combined['FamilySize'].map(lambda s: 1 if s == 1 else 0)
    df_combined['SmallFamily'] = df_combined['FamilySize'].map(lambda s: 1 if 2 <= s <= 4 else 0)
    df_combined['LargeFamily'] = df_combined['FamilySize'].map(lambda s: 1 if 5 <= s <= 8 else 0)
    return df_combined

df_combined = family_size()
df_combined.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize,Singleton,SmallFamily,LargeFamily
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,2,0,1,0
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,2,0,1,0
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1,1,0,0
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,2,0,1,0
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,1,1,0,0


## 2. Creating categories based on Embarkment location
Now, we'll be categorizing the passangers based on the port of embarkation. From the dataset, we know that there are three values in "Embark" column, C(for Cherbourg), Q(for Queenstown) and S(for Southampton). For the missing values in the dataset, we'll assume that these passagers onboarded at the beginning of the journey. Since the Titanic set sail from Southhampton, we'll fill "S" in place of missing values.

In [18]:
#creating function for creating categories based on embarked location
def embarked():
    global df_combined
    df_combined['Embarked'].fillna('S', inplace=True)
    df_dummies = pd.get_dummies(df_combined['Embarked'], prefix='Embarked')
    df_combined = pd.concat([df_combined, df_dummies], axis=1)
    df_combined.drop('Embarked', axis=1, inplace=True)
    return df_combined

df_combined = embarked()
df_combined.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,FamilySize,Singleton,SmallFamily,LargeFamily,Embarked_C,Embarked_Q,Embarked_S
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,2,0,1,0,False,False,True
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,2,0,1,0,True,False,False
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,1,1,0,0,False,False,True
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,2,0,1,0,False,False,True
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,1,1,0,0,False,False,True


## 3. Creating categories based on the cabin of the passangers
From the dataset, we can see that the cabin columns has values in format of "C123", which means the 123th cabin in the C deck(please refer the image). Assuming that passangers from each cabin in a particular deck has equal probabilty of survival, e.g:- a passanger from C123 and C20 each, has same chance of survival. And, a passanger from B83 and B12 each, has same chance of survival, you get the point, we'll remove the cabin number and just keep the deck of the cabin. For missing values of cabin, we'll use "U" for unknown. After getting the deck values for all the passangers, we'll categorize them in deck A, B, C, D, E, F, G, T and U.

In [19]:
#creating function for creating categories based on the cabin of passengers
def cabin():
    global df_combined
    #replacing missing cabin values with U for unknown
    df_combined['Cabin'].fillna('U', inplace=True)
    
    df_combined['Cabin'] = df_combined['Cabin'].map(lambda ca: ca[0])
    #dummy encoding
    cabin_dummies = pd.get_dummies(df_combined['Cabin'], prefix='Cabin')
    df_combined = pd.concat([df_combined, cabin_dummies], axis=1)
    df_combined.drop('Cabin', inplace=True, axis=1)
    return df_combined

df_combined = cabin()
df_combined.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,FamilySize,Singleton,SmallFamily,LargeFamily,Embarked_C,Embarked_Q,Embarked_S,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Cabin_U
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,2,0,1,0,False,False,True,False,False,False,False,False,False,False,False,True
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,2,0,1,0,True,False,False,False,False,True,False,False,False,False,False,False
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,1,1,0,0,False,False,True,False,False,False,False,False,False,False,False,True
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,2,0,1,0,False,False,True,False,False,True,False,False,False,False,False,False
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,1,1,0,0,False,False,True,False,False,False,False,False,False,False,False,True


## 4. Creating categories based on the Age of passangers
Now, we'll segregate passangers based on their age. Since the spread of age property is high and ranges from 0.4 to 80 years, we'll create smaller ranges of Age to categorize passangers. Here, we'he chosen 0(for less than 11), 1(12 to 18), 2(19 to 22), 3(23 to 27), 4(28 to 33), 5(34 to 46), 6(47 to 66), 7(67 and above).

In [20]:
#converting age decimal values to integer
df_combined['Age'] = df_combined['Age'].astype(int)
#Converting the age range into categories
df_combined.loc[df_combined['Age'] <= 11, 'Age'] = 0
df_combined.loc[(df_combined['Age'] > 11) & (df_combined['Age'] <= 18), 'Age'] = 1
df_combined.loc[(df_combined['Age'] > 18) & (df_combined['Age'] <= 22), 'Age'] = 2
df_combined.loc[(df_combined['Age'] > 22) & (df_combined['Age'] <= 27), 'Age'] = 3
df_combined.loc[(df_combined['Age'] > 27) & (df_combined['Age'] <= 33), 'Age'] = 4
df_combined.loc[(df_combined['Age'] > 33) & (df_combined['Age'] <= 46), 'Age'] = 5
df_combined.loc[(df_combined['Age'] > 46) & (df_combined['Age'] <= 66), 'Age'] = 6
df_combined.loc[df_combined['Age'] >= 66, 'Age'] = 7
#distribution
df_combined['Age'].value_counts()

Age
4    442
5    216
3    170
2    138
6    138
1    105
0     91
7      9
Name: count, dtype: int64

## 5. Creating categories based on Fare of passangers
Finally, now we'll tag passangers based on their economic status or the fare they paid for the journey. Again, since the spread of prices is very high and ranges from 0 to 512. So, again we'll segment passangers based on the fare in smaller bins of 0(less than 7), 1(8 to 14), 2(15 to 31), 3(32 to 99), 4(100 to 250), 5(251 and above).

In [21]:
#converting fare to interger values and then into categories
df_combined.loc[df_combined['Fare'] <= 7, 'Fare'] = 0
df_combined.loc[(df_combined['Fare'] > 7) & (df_combined['Fare'] <= 14), 'Fare'] = 1
df_combined.loc[(df_combined['Fare'] > 14) & (df_combined['Fare'] <= 31), 'Fare'] = 2
df_combined.loc[(df_combined['Fare'] > 31) & (df_combined['Fare'] <= 99), 'Fare'] = 3
df_combined.loc[(df_combined['Fare'] > 99) & (df_combined['Fare'] <= 250), 'Fare'] = 4
df_combined.loc[df_combined['Fare'] >= 250, 'Fare'] = 5
df_combined['Fare'] = df_combined['Fare'].astype(int)
#distribution
df_combined['Fare'].value_counts()

Fare
1    605
2    336
3    247
4     67
0     37
5     17
Name: count, dtype: int64

## 6. Creating categories based on the Title of the passangers
Now, it is to categorize based on Title of passangers. This is not a straight-forward value which we can analyze mathmatically, as the title of a person wouldn't mean anything to the iceberg that hit Titanic or to the chilling water that drowned anyone it could. But we know that passangers registered with Mr, Mrs, Master and other royal names would probably be rich and would have been given preference while saving and transferring using life boats. So, we'll categorize them only into Mr, Mrs, Master, Miss and Rare(which contains all other royal and rare titles used by very few passangers).

In [22]:
###Get title from name
common_titles = ["Mr", "Mrs", "Miss", "Master"]
titles = []
for name in df_combined['Name']:
    title = name.split(',')[1].split('.')[0].strip()
    if title in common_titles:
        titles.append(title)
    elif title=="Mlle":
        titles.append("Miss")
    elif title=="Mme":
        titles.append("Mrs")
    else:
        titles.append("Rare")
df_titles = pd.DataFrame(titles, columns=['Titles'])
title_dummies = pd.get_dummies(df_titles['Titles'], prefix='Title')
title_dummies = title_dummies.reset_index(drop=True)
df_combined = df_combined.reset_index(drop=True)
df_combined = pd.concat([df_combined, title_dummies], axis=1)
df_combined.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,FamilySize,Singleton,SmallFamily,LargeFamily,Embarked_C,Embarked_Q,Embarked_S,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Cabin_U,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Rare
0,1,3,"Braund, Mr. Owen Harris",male,2,1,0,A/5 21171,1,2,0,1,0,False,False,True,False,False,False,False,False,False,False,False,True,False,False,True,False,False
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,5,1,0,PC 17599,3,2,0,1,0,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False
2,3,3,"Heikkinen, Miss. Laina",female,3,0,0,STON/O2. 3101282,1,1,1,0,0,False,False,True,False,False,False,False,False,False,False,False,True,False,True,False,False,False
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,5,1,0,113803,3,2,0,1,0,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,True,False
4,5,3,"Allen, Mr. William Henry",male,5,0,0,373450,1,1,1,0,0,False,False,True,False,False,False,False,False,False,False,False,True,False,False,True,False,False


**Creating categories for Male and Female, and dropping few uneccessary columns**

In [23]:
#dummies for gender
gender_dummies = pd.get_dummies(df_combined['Sex'])
df_combined = pd.concat([df_combined, gender_dummies], axis=1)
df_combined.drop('Sex', axis=1, inplace=True)
df_combined.head()

,PassengerId,Pclass,Name,Age,SibSp,Parch,Ticket,Fare,FamilySize,Singleton,SmallFamily,LargeFamily,Embarked_C,Embarked_Q,Embarked_S,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Cabin_U,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Rare,female,male
0,1,3,"Braund, Mr. Owen Harris",2,1,0,A/5 21171,1,2,0,1,0,False,False,True,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,True
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",5,1,0,PC 17599,3,2,0,1,0,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,True,False
2,3,3,"Heikkinen, Miss. Laina",3,0,0,STON/O2. 3101282,1,1,1,0,0,False,False,True,False,False,False,False,False,False,False,False,True,False,True,False,False,False,True,False
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",5,1,0,113803,3,2,0,1,0,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,True,False,True,False
4,5,3,"Allen, Mr. William Henry",5,0,0,373450,1,1,1,0,0,False,False,True,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,True


In [24]:
#dropping Name, Ticket and PassengerId columns
df_combined.drop(['PassengerId',"Name","Ticket"], axis=1, inplace=True)
df_combined.head()

,Pclass,Age,SibSp,Parch,Fare,FamilySize,Singleton,SmallFamily,LargeFamily,Embarked_C,Embarked_Q,Embarked_S,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Cabin_U,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Rare,female,male
0,3,2,1,0,1,2,0,1,0,False,False,True,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,True
1,1,5,1,0,3,2,0,1,0,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,True,False
2,3,3,0,0,1,1,1,0,0,False,False,True,False,False,False,False,False,False,False,False,True,False,True,False,False,False,True,False
3,1,5,1,0,3,2,0,1,0,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,True,False,True,False
4,3,5,0,0,1,1,1,0,0,False,False,True,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,True


## Building and Training the Model

In [25]:
#Separating the train and test dataframes
x_train = df_combined[:891].copy()
x_test = df_combined[891:].copy()
x_test.reset_index(inplace=True, drop=True)
x_train.shape, x_test.shape

((891, 28), (418, 28))

## 1. Neural Network

In [29]:
import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Dense

# Convert data type to float32
x_train = x_train.astype('float32')
y_train = y_train.astype('float32')

# Build the neural network model
nn_model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(x_train.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
nn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
nn_model.fit(x_train, y_train, epochs=1000, batch_size=512)

Epoch 1/1000
2/2 [==============================] - 1s 12ms/step - loss: 0.7311 - accuracy: 0.4366
Epoch 2/1000
2/2 [==============================] - 0s 6ms/step - loss: 0.6880 - accuracy: 0.4893
Epoch 3/1000
2/2 [==============================] - 0s 5ms/step - loss: 0.6571 - accuracy: 0.7138
Epoch 4/1000
2/2 [==============================] - 0s 5ms/step - loss: 0.6328 - accuracy: 0.7183
Epoch 5/1000
2/2 [==============================] - 0s 5ms/step - loss: 0.6137 - accuracy: 0.7071
Epoch 6/1000
2/2 [==============================] - 0s 6ms/step - loss: 0.5986 - accuracy: 0.7003
Epoch 7/1000
2/2 [==============================] - 0s 5ms/step - loss: 0.5859 - accuracy: 0.6891
Epoch 8/1000
2/2 [==============================] - 0s 5ms/step - loss: 0.5734 - accuracy: 0.6981
Epoch 9/1000
2/2 [==============================] - 0s 5ms/step - loss: 0.5610 - accuracy: 0.7262
Epoch 10/1000
2/2 [==============================] - 0s 5ms/step - loss: 0.5495 - accuracy: 0.7419
Epoch 11/1000
2/2 

In [30]:
# 3. Evaluate the model
_, accuracy = nn_model.evaluate(x_train, y_train)
nn_model_accuracy = round(accuracy * 100, 2)

28/28 [==============================] - 0s 1ms/step - loss: 0.2096 - accuracy: 0.9169


In [32]:
# Convert data type to float32
x_test = x_test.astype('float32')

# 4. Make predictions on the test set
y_pred_nn_model = nn_model.predict(x_test)

14/14 [==============================] - 0s 1ms/step


## 2. Random Forest

In [33]:
rf_model = RandomForestClassifier()
rf_model.fit(x_train, y_train)

y_pred_rf_model = rf_model.predict(x_test)

rf_model_accuracy = round(rf_model.score(x_train, y_train)*100, 2)

## 3. Logistic Regression

In [35]:
lr_model = LogisticRegression(max_iter=10000)
lr_model.fit(x_train, y_train)

y_pred_lr_model = lr_model.predict(x_test)

lr_model_accuracy = round(lr_model.score(x_train, y_train)*100, 2)

## 4. K-Nearest Neighbors

In [36]:
knn_model = KNeighborsClassifier(n_neighbors=3)
knn_model.fit(x_train, y_train)

y_pred_knn_model = knn_model.predict(x_test)

knn_model_accuracy = round(knn_model.score(x_train, y_train)*100, 2)

## 5. Gaussian Naive Bayes

In [37]:
gnb_model = GaussianNB()
gnb_model.fit(x_train, y_train)

y_pred_gnb_model = gnb_model.predict(x_test)

gnb_model_accuracy = round(gnb_model.score(x_train, y_train)*100, 2)

## 6. Perceptrons

In [38]:
per_model = Perceptron()
per_model.fit(x_train, y_train)

y_pred_per_model = per_model.predict(x_test)

per_model_accuracy = round(per_model.score(x_train, y_train)*100, 2)

## 7. Support Vector Machine

In [39]:
svc_model = LinearSVC(max_iter=10000)
svc_model.fit(x_train, y_train)

y_pred_svc_model = svc_model.predict(x_test)

svc_model_accuracy = round(svc_model.score(x_train, y_train)*100, 2)

## 8. Decision Tree

In [40]:
dt_model = DecisionTreeClassifier()
dt_model.fit(x_train, y_train)

y_pred_dt_model = dt_model.predict(x_train)

dt_model_accuracy = round(dt_model.score(x_train, y_train)*100, 2)

## 9. XGBoost

In [41]:
#Transforming data into Dmatrix form
d_x_train = xgb.DMatrix(x_train, label=y_train)

In [42]:
#Setting the parameters
param = {
    'eta': 0.5, 
    'max_depth': 16,  
    'objective': 'multi:softprob',  
    'num_class': 3} 

steps = 20  # The number of training iterations

In [43]:
#Training the model
xgb_model = xgb.train(param, d_x_train, steps)
 
y_pred_xgb = xgb_model.predict(d_x_train)
#Converting the prediction intp numpy array
y_pred_xgb_new = np.asarray([np.argmax(line) for line in y_pred_xgb])

In [44]:
xgb_model_accuracy = round(accuracy_score(y_train, y_pred_xgb_new)*100, 2)

## Model Comparison

In [45]:
best_model = pd.DataFrame({
    'Model': ['Neural Network', 'Random Forest','Logistic Regression','K-Nearest Neighbour','Gaussian Naive Bayes','Perceptron','SVM','Decision Tree','XGBoost'], 
    'Result': [nn_model_accuracy, rf_model_accuracy, lr_model_accuracy, knn_model_accuracy, gnb_model_accuracy, per_model_accuracy, svc_model_accuracy, dt_model_accuracy, xgb_model_accuracy ]})
best_model.sort_values(by=['Result'], ascending=False, inplace=True)
best_model.reset_index(inplace=True, drop=True)
fig = go.Figure(data=[go.Table(
    header=dict(values=list(best_model.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[best_model.Model, best_model.Result],
               fill_color='lavender',
               align='left'))
])
fig.show()

## K-Fold Cross Validation

In [50]:
rf_model1 = RandomForestClassifier(random_state=42)
rf_model1.fit(x_train, y_train)
scores = cross_val_score(rf_model1, x_train, y_train, cv=10, scoring='accuracy')

print("Scores:", scores)
print("Mean:", scores.mean())
print("Standard Deviation:", scores.std())

Scores: [0.78888889 0.83146067 0.75280899 0.84269663 0.82022472 0.84269663
 0.83146067 0.78651685 0.84269663 0.83146067]
Mean: 0.8170911360799001
Standard Deviation: 0.02908357266748794


**This score shows that our model has an average accuracy of 82% with a standard deviation of around 3%**

## Feature Importance

In [60]:
importances = pd.DataFrame({'Feature':x_train.columns,'Importance':np.round(rf_model1.feature_importances_,3)})
importances_df = importances.sort_values('Importance', ascending=False).reset_index(drop=True)
importances_plot = importances.sort_values('Importance', ascending=False).set_index('Feature')
importances_plot.head()

,Importance
Feature,
Age,0.132
male,0.121
Title_Mr,0.114
female,0.097
Pclass,0.087


In [58]:
#Plotting the Features importances
fig_bar_feature = go.Figure()
fig_bar_feature.add_trace(go.Bar(y=importances_df['Feature'], x=importances_df['Importance'], orientation='h'))
fig_bar_feature.update_layout(height=800, title_text='Bar chart representing Features Importances',
                       xaxis_title_text='Importance', yaxis_title_text='Features')
fig_bar_feature.show()

From the graph we can see that all the categories after "Title_Rare" and onwards doesn't play an important role in the prediction, hence dropping those categories.

In [61]:
for i in range(23, importances_df.shape[0]):
    column = importances_df['Feature'][i]
    df_combined.drop([column], inplace=True, axis=1)

df_combined.head()

,Pclass,Age,SibSp,Parch,Fare,FamilySize,Singleton,SmallFamily,LargeFamily,Embarked_C,Embarked_Q,Embarked_S,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_U,Title_Master,Title_Miss,Title_Mr,Title_Mrs,female,male
0,3,2,1,0,1,2,0,1,0,False,False,True,False,False,False,False,True,False,False,True,False,False,True
1,1,5,1,0,3,2,0,1,0,True,False,False,False,True,False,False,False,False,False,False,True,True,False
2,3,3,0,0,1,1,1,0,0,False,False,True,False,False,False,False,True,False,True,False,False,True,False
3,1,5,1,0,3,2,0,1,0,False,False,True,False,True,False,False,False,False,False,False,True,True,False
4,3,5,0,0,1,1,1,0,0,False,False,True,False,False,False,False,True,False,False,True,False,False,True


## Training Random Forest Model Again

In [62]:
#Separate the train and test dataframes
x_train = df_combined[:891].copy()
x_test = df_combined[891:].copy()
x_test.reset_index(inplace=True, drop=True)

In [66]:
rf_model_r = RandomForestClassifier()
rf_model_r.fit(x_train, y_train)
y_pred_rf_model_r = rf_model_r.predict(x_test)

accuracy_rf_model_r = round(rf_model_r.score(x_train, y_train)*100, 2)
print("The accuracy after removing least important Features is {}, which is same as before removing the least importance features.".format(accuracy_rf_model_r))
print("Debug: accuracy_rf_model_r =", accuracy_rf_model_r) 

The accuracy after removing least important Features is 92.26, which is same as before removing the least importance features.
Debug: accuracy_rf_model_r = 92.26


## Training Decision Tree Model Again

In [67]:
dt_model_r = DecisionTreeClassifier()
dt_model_r.fit(x_train, y_train)

y_pred_dt_model_r = dt_model_r.predict(x_test)

dt_model_r_accuracy = round(dt_model_r.score(x_train, y_train)*100, 2)
print("The accuracy after removing least important Features is {}, which is same as before removing the least importance features.".format(dt_model_r_accuracy))
print("Debug: accuracy_rf_model_r =", dt_model_r_accuracy) 

The accuracy after removing least important Features is 92.26, which is same as before removing the least importance features.
Debug: accuracy_rf_model_r = 92.26


## Training XGBoost Model Again

In [99]:
# Transforming data into DMatrix form
d_x_train = xgb.DMatrix(x_train, label=y_train)
d_x_test = xgb.DMatrix(x_test)

In [100]:
# Make predictions on the test set
xgb_model_r = xgb.train(param, d_x_train, steps)
y_pred_xgb_model_r = xgb_model_r.predict(d_x_test)
y_pred_xgb_model_r_new = np.asarray([np.argmax(line) for line in y_pred_xgb_model_r])

In [101]:
# Calculate accuracy on training data
y_pred_train = xgb_model_r.predict(d_x_train)
y_pred_train_new = np.asarray([np.argmax(line) for line in y_pred_train])
xgb_model_r_accuracy = round(accuracy_score(y_train, y_pred_train_new) * 100, 2)

In [102]:
print("The accuracy after removing least important Features is {}, which is same as before removing the least importance features.".format(xgb_model_r_accuracy))
print("Debug: accuracy_rf_model_r =", xgb_model_r_accuracy) 

The accuracy after removing least important Features is 92.26, which is same as before removing the least importance features.
Debug: accuracy_rf_model_r = 92.26


## Submission

In [73]:
# Save predictions for submission
submission_df = pd.DataFrame({'PassengerId': test_data['PassengerId'], 'Survived': y_pred_dt_model_r})
submission_df.to_csv('submission.csv', index=False)
print("Submission successfully saved!!!")

Submission successfully saved!!!


In [74]:
# Save predictions for submission
submission_df = pd.DataFrame({'PassengerId': test_data['PassengerId'], 'Survived': y_pred_rf_model_r})
submission_df.to_csv('submission.csv', index=False)
print("Submission successfully saved!!!")

Submission successfully saved!!!


In [103]:
# Save predictions for submission
submission_df = pd.DataFrame({'PassengerId': test_data['PassengerId'], 'Survived': y_pred_xgb_model_r_new})
submission_df.to_csv('submission.csv', index=False)
print("Submission successfully saved!!!")

Submission successfully saved!!!
